In [1]:
import pandas as pd

salm = pd.read_csv('/Users/evelyn/Desktop/salmonella_complete.csv')

salm

,CDCID,GenusName,InitialExposure,LastExposure,TotalCases,Exposurestate,ExposureState,ExposureCounty1,ExposureCounty2,ExposureCounty3,...,Tornado or Funnel Cloud,Median age,Immigrants,RH,SLP-STP,MSXPD-WDSP,Households Median Income,%Food preparation,"%Farming, fishing, and forestry",Population Density
0,6376,Salmonella,2011/1/8,2011/1/28,11,CT,Connecticut,New London,NaN,NaN,...,0.0,39.990137,18789.558900,65.156814,0.447619,6.988889,66931.53973,6.122548,0.181759,410.397085
1,6742,Salmonella,2011/3/9,2011/3/13,51,LA,Louisiana,Avoyelles,NaN,NaN,...,0.0,38.461644,2218.287671,60.937914,4.092000,7.220000,32146.67123,7.557849,2.222345,50.435357
2,6917,Salmonella,2011/3/3,2011/3/13,8,CT,Connecticut,New Haven,NaN,NaN,...,0.0,38.944932,35658.219180,63.142251,10.584849,7.513636,62243.13425,4.815985,0.186321,1419.958055
3,7025,Salmonella,2011/3/5,2011/3/5,18,MD,Maryland,Frederick,NaN,NaN,...,0.0,38.229863,13284.736990,73.553088,38.650000,12.000000,82495.81370,3.902243,0.245475,350.312225
4,7031,Salmonella,2011/3/12,2011/3/24,81,RI,Rhode Island,Bristol,Kent,Newport,...,0.0,41.007808,11548.093150,66.837121,8.009890,6.838462,62365.46301,6.510218,0.270031,1353.479588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,296667,Salmonella,2019/12/14,2019/12/21,8,ID,Idaho,Bingham,NaN,NaN,...,0.0,33.503699,2709.210959,78.268594,164.773750,5.265000,51442.75068,4.973256,5.115897,21.758399
707,296745,Salmonella,2019/8/11,2019/9/9,23,OR,Oregon,Lane,Marion,Multnomah,...,0.0,37.646575,37384.545660,58.764847,6.417333,5.702000,58058.94292,6.434262,1.853684,740.973471
708,296752,Salmonella,2019/8/2,2019/9/28,15,CA,California,San Bernardino,NaN,NaN,...,0.0,33.167123,120422.000000,28.991035,60.776840,8.269617,61237.30137,5.565473,0.387235,106.695409
709,296754,Salmonella,2019/1/22,2019/1/23,5,CA,California,San Francisco,NaN,NaN,...,0.0,38.200000,63777.767120,NaN,NaN,5.625000,111962.19860,5.754939,0.058895,18661.359020


In [5]:
salm.isna().sum()

CDCID                                0
GenusName                            0
InitialExposure                      0
LastExposure                         0
TotalCases                           0
Exposurestate                        0
ExposureState                        0
ExposureCounty1                      0
ExposureCounty2                    635
ExposureCounty3                    657
ExposureCounty4                    673
ExposureCounty5                    682
ExposureCounty6                    688
ExposureCounty7                    696
ExposureCounty8                    699
ExposureCounty9                    701
ExposureCounty10                   702
ExposureCounty11                   704
ExposureCounty12                   704
ExposureCounty13                   707
ExposureCounty14                   708
ExposureCounty15                   708
ExposureCounty16                   708
ExposureCounty17                   709
ExposureCounty18                   709
ExposureCounty19         

In [11]:
!pip install scikit-learn


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [14]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


errors = []

for k in range(1, 20, 2):
    X = salm[['TEMP', 'MAX', 'MIN', 'DEWP', 'SLP', 'STP', 'WDSP', 'MXSPD', 'PRCP', 'SNDP', 'Fog', 'Rain or Drizzle', 'Snow or Ice Pellets', 'Hail', 'Thunder', 'Tornado or Funnel Cloud']]
    X = X.astype(np.float32)
    imputer = KNNImputer(n_neighbors=k)
    imputed = imputer.fit_transform(X)
    df_imputed = pd.DataFrame(imputed, columns = X.columns)
    #DEWP
    X = df_imputed.drop(['DEWP'], axis=1)
    y = df_imputed['DEWP']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    error1 = np.sqrt(mean_squared_error(y_test, preds))
    #SLP
    X = df_imputed.drop(['SLP'], axis=1)
    y = df_imputed['SLP']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    error2 = np.sqrt(mean_squared_error(y_test, preds))
    #STP
    X = df_imputed.drop(['STP'], axis=1)
    y = df_imputed['STP']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    error3 = np.sqrt(mean_squared_error(y_test, preds))
    #
    X = df_imputed.drop(['WDSP'], axis=1)
    y = df_imputed['WDSP']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    error4 = np.sqrt(mean_squared_error(y_test, preds))
    #
    X = df_imputed.drop(['MXSPD'], axis=1)
    y = df_imputed['MXSPD']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    error5 = np.sqrt(mean_squared_error(y_test, preds))
    
    error = error1 + error2 + error3 + error4 + error5
    errors.append({'K': k, 'RMSE': error})
errors

# we take K = 11

[{'K': 1, 'RMSE': 50.78668141750147},
 {'K': 3, 'RMSE': 49.87307805951776},
 {'K': 5, 'RMSE': 48.44466636424078},
 {'K': 7, 'RMSE': 49.51664735429661},
 {'K': 9, 'RMSE': 49.99040173231359},
 {'K': 11, 'RMSE': 48.193975092943965},
 {'K': 13, 'RMSE': 48.99683417657464},
 {'K': 15, 'RMSE': 49.38520274944101},
 {'K': 17, 'RMSE': 49.512649392595236},
 {'K': 19, 'RMSE': 48.93451061222283}]

In [61]:
X = salm[['TEMP', 'MAX', 'MIN', 'DEWP', 'SLP', 'STP', 'WDSP', 'MXSPD', 'PRCP', 'SNDP', 'Fog', 'Rain or Drizzle', 'Snow or Ice Pellets', 'Hail', 'Thunder', 'Tornado or Funnel Cloud']]
X = X.astype(np.float32)
imputer = KNNImputer(n_neighbors=11)
imputed = imputer.fit_transform(X)
df = pd.DataFrame(imputed, columns = X.columns)

In [62]:
df

,TEMP,MAX,MIN,DEWP,SLP,STP,WDSP,MXSPD,PRCP,SNDP,Fog,Rain or Drizzle,Snow or Ice Pellets,Hail,Thunder,Tornado or Funnel Cloud
0,26.261906,32.507938,20.685715,16.178572,1013.557129,1013.109497,8.646031,15.634921,0.037143,0.0000,0.142857,0.095238,0.142857,0.0,0.000000,0.0
1,60.599998,72.388000,47.021999,47.048000,1021.114014,1017.021973,5.146000,12.366000,1.206000,0.0000,0.200000,0.200000,0.000000,0.0,0.200000,0.0
2,38.302273,46.336365,31.220455,26.881819,1023.769714,1013.184875,8.504545,16.018183,0.166439,0.0000,0.068182,0.295455,0.045455,0.0,0.000000,0.0
3,45.450001,59.250000,37.150002,37.500000,1027.199951,988.549988,8.000000,20.000000,0.000000,0.0000,1.000000,1.000000,0.000000,0.0,0.000000,0.0
4,40.438461,48.117310,34.525963,30.299999,1017.727844,1009.717957,7.622116,14.460577,0.027019,0.0000,0.076923,0.067308,0.076923,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,21.462500,30.873751,12.811250,15.766250,1026.632446,861.858765,4.906250,10.171250,0.027500,1.3375,0.000000,0.000000,0.125000,0.0,0.000000,0.0
707,69.174667,85.068665,55.948002,54.119335,1016.067993,1009.650696,4.589334,10.291333,0.034133,0.0000,0.006667,0.233333,0.000000,0.0,0.026667,0.0
708,80.461052,96.849442,65.446785,45.253105,1009.932861,949.156006,5.721858,13.991475,0.001380,0.0000,0.001724,0.055172,0.000000,0.0,0.020690,0.0
709,51.775002,57.450001,48.049999,43.147137,1028.425049,999.217529,4.325000,9.950000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.0


In [65]:
import math
for i in range(len(salm)):
    
    if(pd.isna(salm.loc[i, "DEWP"])):
        salm.loc[i, "DEWP"] = df.loc[i, "DEWP"]
        
    if(pd.isna(salm.loc[i, "SLP"])):
        salm.loc[i, "SLP"] = df.loc[i, "SLP"]
        
    if(pd.isna(salm.loc[i, "STP"])):
        salm.loc[i, "STP"] = df.loc[i, "STP"]
        
    if(pd.isna(salm.loc[i, "WDSP"])):
        salm.loc[i, "WDSP"] = df.loc[i, "WDSP"]
        
    if(pd.isna(salm.loc[i, "MXSPD"])):
        salm.loc[i, "MXSPD"] = df.loc[i, "MXSPD"]
    
    salm.loc[i, "SLP-STP"] = salm.loc[i, "SLP"] - salm.loc[i, "STP"]
    salm.loc[i, "MSXPD-WDSP"] = salm.loc[i, "MXSPD"] - salm.loc[i, "WDSP"]
    t_c = (salm.loc[i, "TEMP"] - 32)/1.8
    d_c = (salm.loc[i, "DEWP"]-32)/1.8
    rh = 100 * (math.exp(17.27 * d_c/(237.7 + d_c))/math.exp(17.27 * t_c/(237.7 + t_c)))
    salm.loc[i, "RH"] = rh
    

In [66]:
salm.isna().sum()

CDCID                                0
GenusName                            0
InitialExposure                      0
LastExposure                         0
TotalCases                           0
Exposurestate                        0
ExposureState                        0
ExposureCounty1                      0
ExposureCounty2                    635
ExposureCounty3                    657
ExposureCounty4                    673
ExposureCounty5                    682
ExposureCounty6                    688
ExposureCounty7                    696
ExposureCounty8                    699
ExposureCounty9                    701
ExposureCounty10                   702
ExposureCounty11                   704
ExposureCounty12                   704
ExposureCounty13                   707
ExposureCounty14                   708
ExposureCounty15                   708
ExposureCounty16                   708
ExposureCounty17                   709
ExposureCounty18                   709
ExposureCounty19         

In [67]:
salm.to_csv('/Users/evelyn/Desktop/Salm_KNN_complete.csv')

In [77]:
noro = pd.read_csv('/Users/evelyn/Desktop/norovirus_complete.csv')

In [78]:
noro.isna().sum()

CDCID                                 0
GenusName                             0
InitialExposure                       0
LastExposure                          0
TotalCases                            0
Exposurestate                         0
ExposureState                         0
ExposureCounty1                       0
ExposureCounty2                    1924
ExposureCounty3                    1937
ExposureCounty4                    1944
ExposureCounty5                    1946
ExposureCounty6                    1948
ExposureCounty7                    1950
ExposureCounty8                    1951
ExposureCounty9                    1951
ExposureCounty10                   1951
ExposureCounty11                   1951
TEMP                                  0
MAX                                   0
MIN                                   0
DEWP                                 18
SLP                                 259
STP                                  67
WDSP                                  4


In [79]:
errors = []

for k in range(1, 20, 2):
    X = noro[['TEMP', 'MAX', 'MIN', 'DEWP', 'SLP', 'STP', 'WDSP', 'MXSPD', 'PRCP', 'SNDP', 'Fog', 'Rain or Drizzle', 'Snow or Ice Pellets', 'Hail', 'Thunder', 'Tornado or Funnel Cloud']]
    X = X.astype(np.float32)
    imputer = KNNImputer(n_neighbors=k)
    imputed = imputer.fit_transform(X)
    df_imputed = pd.DataFrame(imputed, columns = X.columns)
    #DEWP
    X = df_imputed.drop(['DEWP'], axis=1)
    y = df_imputed['DEWP']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    error1 = np.sqrt(mean_squared_error(y_test, preds))
    #SLP
    X = df_imputed.drop(['SLP'], axis=1)
    y = df_imputed['SLP']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    error2 = np.sqrt(mean_squared_error(y_test, preds))
    #STP
    X = df_imputed.drop(['STP'], axis=1)
    y = df_imputed['STP']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    error3 = np.sqrt(mean_squared_error(y_test, preds))
    #WDSP
    X = df_imputed.drop(['WDSP'], axis=1)
    y = df_imputed['WDSP']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    error4 = np.sqrt(mean_squared_error(y_test, preds))
    #MXSPD
    X = df_imputed.drop(['MXSPD'], axis=1)
    y = df_imputed['MXSPD']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    error5 = np.sqrt(mean_squared_error(y_test, preds))
    
    error = error1 + error2 + error3 + error4 + error5
    errors.append({'K': k, 'RMSE': error})
errors

# we take K = 1

[{'K': 1, 'RMSE': 48.2413449923335},
 {'K': 3, 'RMSE': 51.49905350591195},
 {'K': 5, 'RMSE': 50.04760617095574},
 {'K': 7, 'RMSE': 49.12491183280909},
 {'K': 9, 'RMSE': 50.25449458953946},
 {'K': 11, 'RMSE': 50.57534216122303},
 {'K': 13, 'RMSE': 49.71373768673614},
 {'K': 15, 'RMSE': 49.97105549769558},
 {'K': 17, 'RMSE': 50.02728235317462},
 {'K': 19, 'RMSE': 49.48456719349753}]

In [80]:
X = noro[['TEMP', 'MAX', 'MIN', 'DEWP', 'SLP', 'STP', 'WDSP', 'MXSPD', 'PRCP', 'SNDP', 'Fog', 'Rain or Drizzle', 'Snow or Ice Pellets', 'Hail', 'Thunder', 'Tornado or Funnel Cloud']]
X = X.astype(np.float32)
imputer = KNNImputer(n_neighbors=1)
imputed = imputer.fit_transform(X)
df = pd.DataFrame(imputed, columns = X.columns)

In [81]:
for i in range(len(noro)):
    
    if(pd.isna(noro.loc[i, "DEWP"])):
        noro.loc[i, "DEWP"] = df.loc[i, "DEWP"]
        
    if(pd.isna(noro.loc[i, "SLP"])):
        noro.loc[i, "SLP"] = df.loc[i, "SLP"]
        
    if(pd.isna(noro.loc[i, "STP"])):
        noro.loc[i, "STP"] = df.loc[i, "STP"]
        
    if(pd.isna(noro.loc[i, "WDSP"])):
        noro.loc[i, "WDSP"] = df.loc[i, "WDSP"]
        
    if(pd.isna(noro.loc[i, "MXSPD"])):
        noro.loc[i, "MXSPD"] = df.loc[i, "MXSPD"]
    
    noro.loc[i, "SLP-STP"] = noro.loc[i, "SLP"] - noro.loc[i, "STP"]
    noro.loc[i, "MSXPD-WDSP"] = noro.loc[i, "MXSPD"] - noro.loc[i, "WDSP"]
    t_c = (noro.loc[i, "TEMP"] - 32)/1.8
    d_c = (noro.loc[i, "DEWP"]-32)/1.8
    rh = 100 * (math.exp(17.27 * d_c/(237.7 + d_c))/math.exp(17.27 * t_c/(237.7 + t_c)))
    noro.loc[i, "RH"] = rh
    

In [82]:
noro.isna().sum()

CDCID                                 0
GenusName                             0
InitialExposure                       0
LastExposure                          0
TotalCases                            0
Exposurestate                         0
ExposureState                         0
ExposureCounty1                       0
ExposureCounty2                    1924
ExposureCounty3                    1937
ExposureCounty4                    1944
ExposureCounty5                    1946
ExposureCounty6                    1948
ExposureCounty7                    1950
ExposureCounty8                    1951
ExposureCounty9                    1951
ExposureCounty10                   1951
ExposureCounty11                   1951
TEMP                                  0
MAX                                   0
MIN                                   0
DEWP                                  0
SLP                                   0
STP                                   0
WDSP                                  0


In [83]:
noro.to_csv('/Users/evelyn/Desktop/Noro_KNN_complete.csv')